# Mapping of drought to Conflict Data

## Settings

In [39]:

import os
import pandas as pd
import re
import numpy as np
import datetime as dt

In [2]:
path_drought = r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\01_Meteorological_Data\ERA5\04_DIs\HoA'
path_conflict = r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\04_Conflict_Data'
path_mapping = r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\06_Drought_Conflict_Mapping'

## Functions

In [3]:
def DI_conflict_mapping(conflict_df, DI_df, dt, spat_dim='county'):
    DI_df['time'] = DI_df.set_index('time').index.shift(dt, freq='M')
    mapping_df = pd.merge(left = conflict_df, right=DI_df, on=['time', spat_dim])
    mapping_df['year'] = mapping_df.time.dt.year
    mapping_df['conflict_boolean'] = mapping_df.conflict_boolean.replace({True: 1, False: 0})
    return mapping_df

## Mapping of drought data to conflict data on admin-level

In [29]:
# open conflict data file
file_conflict = os.path.join(path_conflict, 'monthly_adm_com_conflict', 'HoA_eventbool.csv')
df_conflict = pd.read_csv(file_conflict, parse_dates=['Unnamed: 0']).rename({'Unnamed: 0': 'time', '0': 'conflict_boolean'}, axis=1)

In [26]:
for DI_name in ['SPI-1', 'SPI-3', 'SPI-6', 'SPI-12', 'SPEI-1', 'SPEI-3', 'SPEI-6', 'SPEI-12']:
    regex = r"([A-Z]{3})-%s-(.*)\.csv"%(DI_name)
    try:
        files = [file for file in os.scandir(os.path.join(path_drought, DI_name[:3], 'stats')) if re.match(regex, file.name)]
    except FileNotFoundError:
        files = [file for file in os.scandir(os.path.join(path_drought, DI_name[:4], 'stats')) if re.match(regex, file.name)]
    for i in range(len(files)):
        file_split = re.match(regex, files[i].name)
        adm1_name = file_split.group(2)
        df_DI_adm1 = pd.read_csv(files[i], parse_dates=['time'])
        df_DI_adm1['county'] = adm1_name
        if i == 0:
            df_DI = df_DI_adm1
        else:
            df_DI = pd.concat([df_DI, df_DI_adm1])
    for dt in np.arange(0,8):
        mapped = DI_conflict_mapping(df_conflict, df_DI.copy(), dt)
        mapped.to_csv(os.path.join(path_mapping, 'adm_%s_lag%i_conflict.csv'%(DI_name, dt)))
        mapped_dry = mapped.copy()
        mapped_dry.iloc[:, 4:-2] = mapped_dry.iloc[:, 4:-2].where(mapped_dry.iloc[:, 4:-2]<0,0)
        mapped_dry.to_csv(os.path.join(path_mapping, 'adm_%s_dry_lag%i_conflict.csv'%(DI_name, dt)))
        mapped_wet = mapped.copy()
        mapped_wet.iloc[:, 4:-2] = mapped_wet.iloc[:, 4:-2].where(mapped_wet.iloc[:, 4:-2]>0,0) 
        mapped_wet.to_csv(os.path.join(path_mapping, 'adm_%s_wet_lag%i_conflict.csv'%(DI_name, dt)))

## Mapping of drought data to conflict data on ethnic group level

In [54]:
file_conflict = os.path.join(path_conflict, 'monthly_ethgroup_com_conflict', 'eventbool.csv')
df_conflict = pd.read_csv(file_conflict, names = ['time', 'eth_group', 'conflict_boolean'], header=0, parse_dates=[0])#.set_index('date_start').stack()).rename({0: 'conflict_boolean'}, axis=1)
df_conflict['time'] = df_conflict['time'].dt.normalize()

In [55]:
for DI_name in ['SPI-1', 'SPI-3', 'SPI-6', 'SPI-12', 'SPEI-1', 'SPEI-3', 'SPEI-6', 'SPEI-12']:
    regex = r"Eth_%s-(.*)\.csv"%(DI_name)
    try:
        files = [file for file in os.scandir(os.path.join(path_drought, DI_name[:3], 'stats', 'Ethnic_group')) if re.match(regex, file.name)]
    except FileNotFoundError:
        files = [file for file in os.scandir(os.path.join(path_drought, DI_name[:4], 'stats', 'Ethnic_group')) if re.match(regex, file.name)]
    for i in range(len(files)):
        file_split = re.match(regex, files[i].name)
        eth_name = file_split.group(1)
        df_DI_eth = pd.read_csv(files[i], parse_dates=['time'])
        df_DI_eth['eth_group'] = eth_name
        if i == 0:
            df_DI = df_DI_eth.copy()
        else:
            df_DI = pd.concat([df_DI, df_DI_eth])
    for dt in np.arange(0,8):
        mapped = DI_conflict_mapping(df_conflict, df_DI.copy(), dt, spat_dim='eth_group')
        mapped.to_csv(os.path.join(path_mapping, 'eth_%s_lag%i_conflict.csv'%(DI_name, dt)))
        mapped_dry = mapped.copy()
        mapped_dry.iloc[:, 3:-2] = mapped_dry.iloc[:, 3:-2].where(mapped_dry.iloc[:, 3:-2]<0,0)
        mapped_dry.to_csv(os.path.join(path_mapping, 'eth_%s_dry_lag%i_conflict.csv'%(DI_name, dt)))
        mapped_wet = mapped.copy()
        mapped_wet.iloc[:, 3:-2] = mapped_wet.iloc[:, 3:-2].where(mapped_wet.iloc[:, 3:-2]>0,0) 
        mapped_wet.to_csv(os.path.join(path_mapping, 'eth_%s_wet_lag%i_conflict.csv'%(DI_name, dt)))